
# 0. Import packages
Keep cell, and leave blank. If PyCharm (IDE) automatically imports any functions then they'll appear in this cell. This allows for easy sorting.

In [ ]:
# Intentionally blank


## 0.1. Clear the workspace

In [ ]:
# Clear all variables in the workspace
%reset -f

## 0.2. Ubermag

In [ ]:
import oommfc as oc
oc.runner.runner

In [ ]:
import discretisedfield as df
import micromagneticmodel as mm
import micromagneticdata as md

## 0.3. My custom modules

In [1]:
#%load_ext autoreload
#%autoreload 2
import custom_image_processing as cip
import custom_system_properties as csp
import custom_dispersion_relations as cdr
import dep.custom_physics_equations as cpe
import dep.damping_absorbing_region as dar
import convert_field_units

ModuleNotFoundError: No module named 'include'

## 0.4. General Python

In [ ]:
%matplotlib module://matplotlib_inline.backend_inline
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display
import time
from jupyter_ui_poll import ui_events  # pump UI events  [oai_citation_attribution:0‡Stack Overflow](https://stackoverflow.com/questions/63784503/make-python-script-wait-for-button-click-ipywidgets)

# 1. Define the overall system

Set your data and figure output directories; used for saving the simulation data and figures, respectively. The `json_params` dictionary is used to store parameters that will be passed to the JSON file.

In [ ]:
data_output = '/Users/cameronmceleney/Data/2025-04-18/Ubermag_data'
figs_output = '/Users/cameronmceleney/Data/2025-04-18/Ubermag_output'

OOMMF will automatically generate a JSON file containing most simulation parameters when a driver (`oommfc.Driver`) is executed. However, many parameters used to define customised fields (`df.Field`) will only appear, if at all, in binary-encoded files.

To log these parameters, we can piggyback on OOMMF's auto-generated JSON by passing a dictionary of these parameters to the final driver.

In [ ]:
json_params = {}

## 1.1. Container

- `system.name` will be used extensively for naming output files and figures. Ensure it is descriptive and key-search friendly.
- Assume there is no temperature `T` to consider.

In [ ]:
system = mm.System(name='dmi_paper_validation',
                   T=0)

I prefer to use my own custom system properties class as this allows me to define dimensions, cell sizes, and other properties in a single place prior to declaring any region (`df.Region`) or mesh (`df.Mesh`) instances. For me, `csp.SystemProperties` improve code readability.

> To only use functionality defined by Ubermag one must first define a mesh and then a region. The dimensions can be extracted from either, while the cell size is a property of the mesh.

In [ ]:
# System mesh cell size
main_dims = ('x', 'y', 'z')
main_cellsize = (2e-9, 1e-9, 12e-9)
main_units = ('m', 'm', 'm')

## 1.2. Material regions

### 1.2.1. Subregions
One could solely use the `domain` container we just created to also hold these subregions. For demonstration purposes more than anything else, we first create a second, separate container for the subregions.

While each subregion is separated define, we are somewhat able to automate the process by letting one subregion be the base for the next.

In [ ]:
# system_subregions = csp.MyRegions(system.name + "_subregions")

In [ ]:
def scaling_function(input_region: df.Region,
                     scale_factor: int,
                     cellsize) -> df.Region:

    tiny = df.Region(p1=(input_region.pmax[0],
                                  input_region.pmin[1],
                                  input_region.pmin[2]),
                              p2=(input_region.pmax[0] + cellsize[0],
                                  input_region.pmax[1],
                                  input_region.pmax[2]),
                              )

    output_region = tiny.scale(factor=(scale_factor, 1, 1),
                               reference_point=tiny.pmin,
                               inplace=False
                               )

    for i in range(3):
        output_region.pmin[i] = round(output_region.pmin[i], 9)
        output_region.pmax[i] = round(output_region.pmax[i], 9)

    return output_region


In [ ]:
fixedLhs = df.Region(p1=(0, 0, 0),
                     p2=main_cellsize,
                     dims=main_dims,
                     units=main_units)

dampingLhs = scaling_function(fixedLhs, scale_factor=300, cellsize=main_cellsize)

freeLhs = scaling_function(dampingLhs, scale_factor=2000, cellsize=main_cellsize)

gradientLhs = scaling_function(freeLhs, scale_factor=1000, cellsize=main_cellsize)

driven = scaling_function(gradientLhs, scale_factor=60, cellsize=main_cellsize)

gradientRhs = scaling_function(driven, scale_factor=1000, cellsize=main_cellsize)

freeRhs = scaling_function(gradientRhs, scale_factor=2000, cellsize=main_cellsize)

dampingRhs = scaling_function(freeRhs, scale_factor=300, cellsize=main_cellsize)

fixedRhs = scaling_function(dampingRhs, scale_factor=1, cellsize=main_cellsize)

### 1.2.2. Primary region

All models must consistent of a primary `df.Region` from which the `df.Mesh` can be determined. Now, we set up the container for this primary region which must span all of the subregions we're requesting.

> The command `%%capture` is a Jupyter magic command that suppresses the output of the cell. Comment it out to see the cell's output.

In [ ]:
main_region = df.Region(p1=fixedLhs.pmin,
                        p2=fixedRhs.pmax)

## 1.3. Define the overall mesh

In [ ]:
main_mesh = df.Mesh(region=main_region,
                    cell=main_cellsize,
                    subregions={'fixedLhs': fixedLhs,
                                'dampingLhs': dampingLhs,
                                'freeLhs': freeLhs,
                                'gradientLhs': gradientLhs,
                                'driven': driven,
                                'gradientRhs': gradientRhs,
                                'freeRhs': freeRhs,
                                'dampingRhs': dampingRhs,
                                'fixedRhs': fixedRhs
                                }
                    )

In [ ]:
main_mesh.subregions

It's always a good idea to check that the mesh has been created correctly. The `mpl` method of the mesh object allows us to visualise the mesh and subregions. Note that the proportions of the mesh may not be accurately represented in the plot. The `box_aspect` argument can be tweaked as required to improve the visualisation.

In [ ]:
main_mesh.mpl.subregions(figsize=(4, 4),
                         multiplier=1e-9,
                         box_aspect=(40, 1, 4),
                         show_region=True)

In [ ]:
from src import UbermagInterface
designer = UbermagInterface(cellsize=(2e-9, 1e-9, 1e-9), show_borders=True)

In [ ]:
designer.mesh

## 1.4. Boundary conditions

In [ ]:
main_mesh.bc = 'x'

evolver_fixed_spins_list = ['']#['fixedLhs', 'fixedRhs']

In [ ]:
if any(evolver_fixed_spins_list) and main_mesh.bc:
    raise ValueError("[evolver_fixed_spins_list] and [system_region.mesh.bc] cannot both be specified")


# 2. Define the system's starting magnetisation state

## 2.1. Saturisation magnetisation, $M_{s}$

In [ ]:
sat_mag = 8e5  # [A/m]

In [ ]:
json_params['Saturation fields [A/m]'] = str(sat_mag)

## 2.2. (As needed) Create a mask for the magnetisation of the primary region

Create a mask for the primary region. Cells where $M_{s} = 0$ will be treated as non-magnetic **only if** we pass `valid='norm'` as an argument when defining `system.m`.

If `valid='norm'` is not used, then we must ensure that all other definitions, such as the regions defining $\mathbf{m}_\text{0}$ **exactly** match those used in the mask for $M_{s}$. In this example, failure to do so would lead to cells being initialised with unnormalised magnetisations. This almost always leads to garbage, and can be a very subtle error to debug.

In [ ]:
def sat_mag_func(pos):
    valid_regions = [main_region.region]

    if any(pos in region for region in valid_regions):
        return sat_mag
    else:
        return 0.0

## 2.3. Initial magnetisation, $\mathbf{m}_{0}$

In [ ]:
def m_init_func(pos):
    valid_regions = [fixedLhs, fixedRhs]

    if any(pos in region for region in valid_regions):
        return (0.0, 0.0, 1.0)
    else:
        return (0.0, 0.0, 1.0)

## 2.4. Define the system's magnetisation field

In [ ]:
system.m = df.Field(
    mesh=main_mesh,
    value=(0.0, 0.0, 1.0),
    norm=sat_mag,
    #valid='any',
    nvdim=3
)

> From this point on temporary containers such as `system_region` are no longer needed. We can now use the `system` container to access the mesh, regions, and subregions. It is a good idea to remove these temporary containers to avoid confusion. The only real use-case for these containers later in the code is to check that `system` has been correctly updated.

# 3. Define the energy equation

We will define the energy equation for our micromagnetic system in Ubermag's container `micromagneticmodel.Energy()`.

> Ensure that variable names are descriptive and key-search friendly. When errors occur Ubermag, OOMMF, and Python messages are spewed out together; debugging is nightmare. This is why `sat_mag` is used instead of `Ms`.




In [ ]:
system.energy = 0

## 3.1. Quick access

To save scrolling deep into the Notebook, I like to forward declare my core material parameters.

Not yet implemented properly; temporary solution below.

In [ ]:
from Uberwidgets.user_interface.builder import build_simulation_ui

# my_energy = EnableEnergyTerms()
# my_sim_params = EnableSimulationParameters()

energy_model, sim_model, _, _, _ = build_simulation_ui()

In [ ]:
energy_model.symbol('anisotropy')

## 3.2. Static Zeeman field

In [ ]:
zeeman_static_dict = {'chain': (0.0, 0.0, 0.16),
                      'driven': (0.0, 0.0, 0.4),
                      }

for key, vals in zeeman_static_dict.items():
    zeeman_static_dict[key] = convert_field_units.ampere_per_metre(vals)

json_params['chain'] = zeeman_static_dict['chain']
json_params['driven'] = zeeman_static_dict['driven']

In [ ]:
zeemanGradientL = dar.LinearGradientField(
    grad_region=gradientLhs,
    p1=gradientLhs.pmin[0],
    p2=gradientLhs.pmax[0],
    field_strength_left=zeeman_static_dict['chain'],
    field_strength_right=zeeman_static_dict['driven'],
    cell_size=main_cellsize
)

zeeman_drive_only = dar.UniformFieldStrength(
    region=driven,
    uniform_field_strength=zeeman_static_dict['driven']
)

zeemanGradientR = dar.LinearGradientField(
    grad_region=gradientRhs,
    p1=gradientRhs.pmin[0],
    p2=gradientRhs.pmax[0],
    field_strength_left=zeeman_static_dict['driven'],
    field_strength_right=zeeman_static_dict['chain'],
    cell_size=main_cellsize
)

zeeman_functors = { 'gradientL': zeemanGradientL,
                    'driven_only': zeeman_drive_only,
                    'gradientR': zeemanGradientR,
                    }

zeeman_profiles = { 'gradientLhs': gradientLhs,
                    'driven': driven,
                    'gradientRhs': gradientRhs,
                    }

static_zeeman_full = dar.CompositeFieldStrength(
    profiles=list(zeeman_functors.values()),
    field_strength_bulk=zeeman_static_dict['chain']
)

zeeman_mesh = df.Mesh(
    region    = main_region,
    cell      = main_mesh.cell,
    subregions=zeeman_profiles
)

static_zeeman_field = df.Field(
    mesh  = zeeman_mesh,
    nvdim = 3,
    value = static_zeeman_full
)

In [ ]:
zeeman_static_strength = [0.0, 0.0, 0.65]  # [T]
zeeman_static_strength = convert_field_units.ampere_per_metre(zeeman_static_strength)

In [ ]:
if energy_model.static_zeeman:
    system.energy += mm.Zeeman(H=static_zeeman_field,
                               name='static')

## 3.3. Driving Zeeman field

In [ ]:
driving_field_freq = 18.00 * 1e9

# Stepsize at which equations are evaluated
stepsize = 1e-12  # [s]

driving_field_strength = [5e-3, 0, 0]  # [T]

In [ ]:
driving_field_strength = convert_field_units.ampere_per_metre(list(driving_field_strength))

json_params['h0_x [A/m]'] = str(driving_field_strength[0])
json_params['h0_y [A/m]'] = str(driving_field_strength[1])
json_params['h0_z [A/m]'] = str(driving_field_strength[2])

If not already set, then we now define where the line antennae are positioned. Update meshes, and print new mesh to visually check change(s).

### 3.3.1. Strength of the driving field

For true control over the implementation of $\mathbf{h}_\text{d}$ we must define a customised field (`df.Field`). This determines whether a cell is located in a driven region.

In [ ]:
def h0_func(pos):
    if pos in driven:
        return driving_field_strength
    else:
         # No drive outside these regions
        return (0, 0, 0)

For clarity when reading the output JSON files, we create a separate mesh for the driving field.

In [ ]:
driving_mesh = df.Mesh(region=main_region,  # use Domain to show both domain explicitly use the same region
                       cell=main_cellsize,  # Use mesh.cell to show the coupling between the two domain
                       subregions={'main_region': main_region,
                                   'driven': driven
                                   }
                       )

We can now create a field for $\mathbf{h}_\text{d}$, and add it to the respective energy term.

In [ ]:
h0_field = df.Field(mesh=driving_mesh,
                    nvdim=3,
                    value=h0_func)

In [ ]:
driving_mesh.mpl.subregions(figsize=(4, 4),
                            multiplier=1e-9,
                            box_aspect=(40, 1, 4),
                            show_region=False)

If you are using `tcl_strings` then you **must** ensure that `mm.Zeeman.name` is identical to the name passed to the tcl script.

In [ ]:
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.graph_objs import FigureWidget

# --- 1. Define your subregions (example placeholders) ---
# In practice, mesh_data[name] should give you:
#   - vertices: (N×3) array of floats
#   - i,j,k: index triples defining triangles
mesh_data = {
    'Region A': {
        'vertices': [[0,0,0], [1,0,0], [1,1,0], [0,1,0]],
        'i': [0,1,2, 0,2,3],
        'j': [1,2,3, 1,3,0],
        'k': [2,3,0, 2,0,1],
    },
    'Region B': {
        'vertices': [[1,0,0], [2,0,0], [2,1,0], [1,1,0]],
        'i': [0,1,2, 0,2,3],
        'j': [1,2,3, 1,3,0],
        'k': [2,3,0, 2,0,1],
    },
    # … add as many regions as you need …
}

# --- 2. Build the 3D figure with one trace per region ---
fig = FigureWidget(
    layout=go.Layout(
        scene=dict(aspectmode='data'),
        width=600, height=400
    )
)

for name, data in mesh_data.items():
    verts = data['vertices']
    fig.add_trace(go.Mesh3d(
        x=[v[0] for v in verts],
        y=[v[1] for v in verts],
        z=[v[2] for v in verts],
        i=data['i'], j=data['j'], k=data['k'],
        name=name,
        opacity=0.5
    ))

# --- 3. Prepare ipywidgets ---
# Header: constant-parameter toggles
const_exch = widgets.Checkbox(
    description='Constant exchange stiffness', value=False
)
exch_value = widgets.FloatText(
    description='A (J/m)', disabled=True
)
def toggle_const(change):
    exch_value.disabled = not change['new']
const_exch.observe(toggle_const, names='value')

header = widgets.HBox([const_exch, exch_value])

# Left: region selector & parameter fields
region_dropdown = widgets.Dropdown(
    options=list(mesh_data.keys()),
    description='Region:'
)
# Placeholder for more parameters (e.g. zeeman field)
param_box = widgets.VBox([region_dropdown])

# Bottom: action buttons
btn_instantiate = widgets.Button(description='Instantiate all')
btn_update      = widgets.Button(description='Update')

bottom = widgets.HBox([btn_instantiate, btn_update])

# --- 4. Hook up click events on the 3D plot ---
def handle_click(trace, points, state):
    # trace.name is the region title
    region_dropdown.value = trace.name

for trace in fig.data:
    trace.on_click(handle_click)

# --- 5. Layout everything ---
ui = widgets.VBox([
    header,
    widgets.HBox([param_box, fig]),
    bottom
])

display(ui)

# --- 6. (Later) Define callbacks for instantiate/update ---
def on_instantiate(b):
    # gather all widget values and build your Ubermag system...
    print("Instantiate with:")
    print("  constant A?", const_exch.value,
          "A =", exch_value.value)
    print("  region-specific params for", region_dropdown.value)
btn_instantiate.on_click(on_instantiate)

def on_update(b):
    # update only the selected region's parameters
    print("Update region", region_dropdown.value)
btn_update.on_click(on_update)

In [ ]:
energy_zeeman_drive = mm.Zeeman(H=h0_field,
                                name='pump',
                                func='sin',
                                f=driving_field_freq,
                                t0=0*stepsize)

## 3.4. Dzyaloshinskii-Moriya interaction (DMI)

In [ ]:
D_ij = 2.5e-3 # [J/m^2]

D_ij_gradient = {'chain': 0.01e-3, 'driven': D_ij, 'lower': 0.01e-3, 'upper': D_ij}
dmi_subregion_mapping = {}

In [ ]:
json_params['D_ij [J/m^2]'] = D_ij_gradient['driven']
json_params['has_energy.dmi.constant'] = energy_model.dmi['constant']
json_params['has_energy.dmi.dict'] = energy_model.dmi['dictionary']
json_params['has_energy.dmi.subregions'] = energy_model.dmi['subregions']

### 3.4.1. Option to manually define DMI for each region

In [ ]:
def generate_dmi_dict(region_with_subregions: csp.MyRegions,
                      DMI_constant: float):

    dmi_region = {}
    dmi_subregions = {key : DMI_constant for key in region_with_subregions.subregions.keys()}

    last_region_seen = 'entire'

    for key in dmi_subregions.keys():
        dmi_region[f'{last_region_seen}:{key}'] = DMI_constant
        last_region_seen = key
        dmi_region[key] = DMI_constant
    dmi_region[f'{last_region_seen}:entire'] = DMI_constant

    return dmi_region

In [ ]:
if energy_model.dmi['dictionary']:
    dmi_subregion_mapping = generate_dmi_dict(system.m.mesh, D_ij_gradient['chain'])
    json_params['D_ij (chain) [J/m^2]'] = D_ij_gradient['chain']
    json_params['D_ij (driven) [J/m^2]'] = D_ij_gradient['driven']


Manually set specific subregions' DMI values.

In [ ]:
if energy_model.dmi['dictionary']:
    dmi_subregion_mapping['gradientLhs:driven'] = D_ij_gradient['driven'] / 2
    dmi_subregion_mapping['driven'] = D_ij_gradient['driven']
    dmi_subregion_mapping['driven:gradientRhs'] = D_ij_gradient['driven'] / 2

### 3.4.2. Option to apply gradients to region(s) with DMI

In [ ]:
num_dmi_subdivisions = 500 #round((gradientLhs.pmax[0] - gradientLhs.pmin[0]) / main_cellsize[0])

Make sure to update the JSON with these values as the MIF file will be difficult to read.

In [ ]:
if energy_model.dmi['subregions']:
    for key, value in D_ij_gradient.items():
        json_params[key] = value

First we need to extract the DMI values for the subregions that have already been defined. We start with the left-hand side of the system. Note that we exclude the gradientLhs subregion itself.

In [ ]:
dmi_lhs_before_gradient = {}

if energy_model.dmi['subregions']:
    for key in system.m.mesh.subregions.keys():
        if key == 'gradientLhs':
            break
        dmi_lhs_before_gradient[key] = D_ij_gradient['chain']

Now we move to create a dictionary for the right-hand side of the system.

In [ ]:
dmi_rhs_after_gradient = {}

if energy_model.dmi['subregions']:
    append_to_dmi_rhs = False
    for key in system.m.mesh.subregions.keys():
        if key == 'gradientRhs':
            append_to_dmi_rhs = True
            continue
        if append_to_dmi_rhs:
            dmi_rhs_after_gradient[key] = D_ij_gradient['chain']

We are now able to begin discretising the gradient regions to fake a DMI field. We start with the left-hand size.

In [ ]:
dmi_lhs_mapping = {}
if energy_model.dmi['subregions']:
    new_mesh, dmi_lhs_mapping = csp.subdivide_region_new(
        mesh            = main_mesh,
        region          = gradientLhs,
        value_min       = D_ij_gradient['lower'],
        value_max       = D_ij_gradient['upper'],
        n_subdivisions  = num_dmi_subdivisions,
        axis            = 'x',
        name_root       = 'gradientLhsDmiSlice',
        remove_parent   = True
    )
    main_mesh = new_mesh
    system.m = df.Field(
        mesh=main_mesh,
        value=m_init_func,
        norm=sat_mag,
        #valid='norm',
        nvdim=3
    )

Make sure to include the DMI value within the driving region!

In [ ]:
dmi_center = {}
if energy_model.dmi['subregions']:
    dmi_center = {'driven': D_ij_gradient['driven']}

Now the DMI values for the right-hand side gradient region.

In [ ]:
dmi_rhs_mapping = {}
if energy_model.dmi['subregions']:
    new_mesh, dmi_rhs_mapping = csp.subdivide_region_new(
        mesh            = main_mesh,
        region          = gradientRhs,
        value_min       = D_ij_gradient['upper'],
        value_max       = D_ij_gradient['lower'],
        n_subdivisions  = num_dmi_subdivisions,
        axis            = 'x',
        name_root       = 'gradientRhsDmiSlice',
        remove_parent   = True
    )
    main_mesh = new_mesh
    system.m = df.Field(
        mesh=main_mesh,
        value=m_init_func,
        norm=sat_mag,
        #valid='norm',
        nvdim=3
    )

This is the main trick! By adding the dictionaries as arguments to `csp.add_inter_subregion_values` in the correct order we can automatically generate a dictionary in the form required by Ubermag.

In [ ]:
if energy_model.dmi['subregions']:
    dmi_subregion_mapping = csp.add_inter_subregion_values(dmi_lhs_before_gradient,
                                                           dmi_lhs_mapping,
                                                           dmi_center,
                                                           dmi_rhs_mapping,
                                                           dmi_rhs_after_gradient,
                                                           dmi_chain_left=D_ij_gradient['chain'],
                                                           dmi_chain_right=D_ij_gradient['chain'])

### 3.4.3. Check DMI flags and include energy term

In [ ]:
if energy_model.dmi['constant']:
    system.energy += mm.DMI(D=D_ij,
                            crystalclass='Cnv_y')
elif energy_model.dmi['dictionary']:
    # Build the symmetric version:
    D_complete = {}
    for key, val in dmi_subregion_mapping.items():
        D_complete[key] = val
        if ':' in key:
            a, b = key.split(':', 1)
            D_complete[f'{b}:{a}'] = val

    dmi_subregion_mapping = D_complete

    system.energy += mm.DMI(D=dmi_subregion_mapping,
                            crystalclass='Cnv_y')
elif energy_model.dmi['subregions']:

    system.energy += mm.DMI(D=dmi_subregion_mapping,
                        crystalclass='Cnv_y')
else:
    raise ValueError('DMI not set as constant, dict, or subregions. Please check your code.')

### 3.4.4. Check for chiral texture

Our larger dispersion relations' proven validity is predicated on the DMI not being strong enough to alter the ground  of the magnetic configuration. This is because the ground state is no longer a single magnetic domain, and instead a magnetic texture.

Note that our dispersion relations **may** remain valid even if this condition fails, but this is not guaranteed. Suggested fixes include reducing the strength of `D_ij`, or increasing the thickness of the system along its symmetry-breaking axis.

> This equation is marked Eq. (20) in the paper *moon2013spin*.

In [ ]:
D_ij_threshold = np.sqrt(
    ((2 * 1.3e-11) / (mm.consts.mu0 * sat_mag))
    * (2 * max(zeeman_static_strength)
     + sat_mag
     + 2 * np.sqrt(
                max(zeeman_static_strength)
                * (max(zeeman_static_strength) + sat_mag)
            )
     )
)

print(f"DMI threshold: {D_ij_threshold:.2e} J/m^2. True when A = {1.3e-11}")

In [ ]:
if ((2 * D_ij) / (mm.consts.mu0 * sat_mag)) > D_ij_threshold:
    raise ValueError(f"The DM interaction was set as {D_ij} [J/m^2], "
                     f"but the DMI threshold is {D_ij_threshold} J/m^2.")


## 3.5. Heisenberg exchange interaction

In [ ]:
exchange_stiffness = 1.3e-11 # [J/m]

In [ ]:
if energy_model.exchange:
    system.energy += mm.Exchange(A=exchange_stiffness)

### 3.5.1. Check the exchange length

Check if the selected cell and system dimensions are appropriate for the exchange length and helical length. Failed tests should prevent the simulation from running.

In [ ]:
# # Temporary check to catch a common mistake of mine during these simulations
# if main_cellsize[0] != 1e-9 and exchange_stiffness == 1.3e-11:
#     raise ValueError('Exchange stiffness has likely not been rescaled; your cell size along the x-axis is not 1 nanometer! Was this intentional?')

In [ ]:
exchange_length = np.sqrt((2 * exchange_stiffness) / (mm.consts.mu0 * sat_mag ** 2))

helical_length = (4 * np.pi * exchange_stiffness) / np.abs(D_ij) if D_ij != 0 else np.NZERO

helicity_strength = ((D_ij ** 2) / (2 * exchange_stiffness))

if energy_model.static_zeeman and 0 < max(zeeman_static_strength) < helicity_strength:
    has_helicity_overcome_zeeman = True
else:
    has_helicity_overcome_zeeman = False

In [ ]:
json_params['Exchange length [nm]'] = str(exchange_length * 1e9)
json_params['Helical length [nm]'] = str(helical_length * 1e9)
json_params['Max helicity [kA/m]'] = str(helicity_strength * 1e-3)

In [ ]:
if exchange_length < main_cellsize[0]:
    raise ValueError('Exchange length is smaller than the cell size!')

if helical_length >= main_region.pmax[0]:
    raise ValueError('Helical length is greater than the system length!')

if has_helicity_overcome_zeeman:
    raise ValueError(f'System is likely unstable! Helicity [{helicity_strength:.2f} A/m] is greater than the equilibrium fields [{max(zeeman_static_strength):.2f} A/m]!')

In [ ]:
print(f"Exchange length: {exchange_length * 1e9:.2f} [nm]",
      f"\nHelical length: {helical_length * 1e9:.2f} [nm]",
      f"\nMax helicity: {helicity_strength * 1e-3: .2f} [kA/m]",
      )

## 3.6. Dipole-dipole interaction (demag)

In [ ]:
if energy_model.demag:
    system.energy += mm.Demag()#asymptotic_radius=16)

## 3.7. Uniaxial anisotropy

In [ ]:
K_1 = 6.2e4  # [J/m^3]
K_2 = 0e5  # [J/m^3]

# anisotropy axis
aniso_axis = (0, 0, 1)

In [ ]:
if energy_model.anisotropy:
    system.energy += mm.UniaxialAnisotropy(K1=K_1,
                                           K2=K_2,
                                           u=aniso_axis)

## 3.8. Check our energy equation

In [ ]:
system.energy

# 4. Define the dynamics equation

In [ ]:
system.dynamics = 0

## 4.1. Gilbert damping

In [ ]:
alpha_bulk = 0.001
alpha_driven = 0.01
alpha_interfacial = 0.001

In [ ]:
json_params['alpha_bulk'] = str(alpha_bulk)
json_params['alpha_driven'] = str(alpha_driven)
json_params['alpha_interfacial'] = str(alpha_interfacial)

### 4.1.1. Create the damping regions via functors

In [ ]:
absorbLhs = dar.AbsorbingExponentialAlpha(
    region=dampingLhs,
    edge_width=dampingLhs.pmax[0] - dampingLhs.pmin[0],
    alpha_bulk=alpha_bulk
)

gradientL = dar.LinearGradientAlpha(
    grad_region=gradientLhs,
    p1=gradientLhs.pmin[0],
    p2=gradientLhs.pmax[0],
    alpha_left=alpha_bulk,
    alpha_right=alpha_driven,
    cell_size=main_cellsize
)

drive_only = dar.DrivenRegionAlpha(
    region=driven,
    alpha_driven=alpha_driven
)

gradientR = dar.LinearGradientAlpha(
    grad_region=gradientRhs,
    p1=gradientRhs.pmax[0],
    p2=gradientRhs.pmin[0],
    alpha_left=alpha_driven,
    alpha_right=alpha_bulk,
    cell_size=main_cellsize
)

absorbRhs = dar.AbsorbingExponentialAlpha(
    region=dampingRhs,
    edge_width=dampingRhs.pmax[0] - dampingRhs.pmin[0],
    alpha_bulk=alpha_bulk,
    reverse=True
)

### 4.1.2. Create containers for the damping regions' information
We need a dictionary of the regions so we can define a submesh. We also need a list of the functors so we can obtain a single, unified function that can be passed to the damping field.


In [ ]:
damping_functors = {'absorbLhs': absorbLhs,
                    'gradientL': gradientL,
                    'driven_only': drive_only,
                    'gradientR': gradientR,
                    'absorbRhs': absorbRhs
                    }

damping_profiles = {'dampingLhs': dampingLhs,
                    'gradientLhs': gradientLhs,
                    'driven': driven,
                    'gradientRhs': gradientRhs,
                    'dampingRhs': dampingRhs
                    }

### 4.1.3. Create the damping field

First we obtain a single callable function for the damping field to invoke.

In [ ]:
alpha_full = dar.CompositeAlpha(
    profiles=list(damping_functors.values()),
    alpha_bulk=alpha_bulk
)

To improve readability of the output JSON files, we create a separate mesh for the damping field. This is not strictly necessary, but it helps to clarify the output files.

In [ ]:
damping_mesh = df.Mesh(
    region    = main_region,
    cell      = main_mesh.cell,
    subregions=damping_profiles
)

Finally, we are able to define the `df.Field` for damping, and add this to the system's dynamics.

In [ ]:
alpha_field = df.Field(
    mesh  = damping_mesh,
    nvdim = 1,
    value = alpha_full   # your CompositeAlpha(pos) → α
)

In [ ]:
system.dynamics += mm.Damping(alpha=alpha_field)

### 4.1.4. Checks

In [ ]:
if system.m.mesh.bc:
    if isinstance(system.dynamics.damping.alpha, df.field.Field):
        raise ValueError(f"The damping field is a function of position, but the mesh has periodic boundary conditions. This may be an error!")

elif any(evolver_fixed_spins_list):
    if isinstance(system.dynamics.damping.alpha, float | int):
        raise ValueError(f'The damping field is a constant value of {system.dynamics.damping.alpha}, but the mesh does not have periodic boundary conditions. This is likely an error!')

## 4.2. Precession

Gyromagnetic ratio, $\gamma_{0}$.

If value is provided here then one MUST use gamma_G (not gamma_LL) when defining subsequent evolvers

In [ ]:
gamma0 = 28.2e9 * 2 * np.pi  # [rad*Hz/T]

In [ ]:
system.dynamics += mm.Precession(gamma0=gamma0*mm.consts.mu0)

## 4.3. Check our expression

In [ ]:
system.dynamics

In [ ]:
energy_model.dmi

## 4.4. Check our dispersion relation

In [ ]:
# Set range for wavevectors (not beyond first Brillouin zone at $\pi/lattice_constant$
kmax = 0.8 * 1e9
mesh_numcells = [int(num_cells * cell_size * 1e9) for num_cells, cell_size in zip(system.m.mesh.n, main_cellsize)]
ks = np.linspace(1e-10, kmax, round(mesh_numcells[0] / 2))
ks_n = np.linspace(-kmax, 1e-10, round(mesh_numcells[0] / 2))

# Plot figure
fig1, ax1 = plt.subplots(figsize=(4.5, 3),
                         nrows=1,
                         ncols=1,
                         layout='constrained')

system_prop = csp.SystemProperties(length=main_region.pmax[0],
                                   width=main_region.pmax[1],
                                   thickness=main_region.pmax[2])

for zeeman_strength_key, zeeman_strength_val in zeeman_static_dict.items():
    dmi_strength = D_ij_gradient[zeeman_strength_key]

    oms_g_ua = cdr.Omega_generalised_with_ua(system_prop,
                                             max(zeeman_strength_val), sat_mag, exchange_stiffness, dmi_strength,
                                             ks, main_cellsize[0],
                                             K_1, K_2, aniso_axis,
                                             gamma0 / (2 * np.pi * 1e9), p=1,
                                             has_demag=energy_model.demag,
                                             has_dmi=any(energy_model.dmi.values()),
                                             has_aniso=energy_model.anisotropy)

    oms_g_ua_n = cdr.Omega_generalised_with_ua(system_prop,
                                               max(zeeman_strength_val), sat_mag, exchange_stiffness, dmi_strength,
                                               ks_n, main_cellsize[0],
                                               K_1, K_2, aniso_axis,
                                               gamma0 / (2 * np.pi * 1e9), p=1,
                                               has_demag=energy_model.demag,
                                               has_dmi=any(energy_model.dmi.values()),
                                               has_aniso=energy_model.anisotropy)

    kittel_freq = min(min(oms_g_ua), min(oms_g_ua_n))

    if zeeman_strength_key == 'chain':
        plot_colour = 'purple'
    else:
        plot_colour = 'red'

    ax1.plot(ks * 1e-9, oms_g_ua,
             lw=2, ls='-',
             color=plot_colour, alpha=0.75,
             label=r'$f_{0}$'+f'({zeeman_strength_key}): {kittel_freq: .2f}',
             zorder=1.2)

    ax1.plot(ks_n * 1e-9, oms_g_ua_n,
             lw=2, ls='-',
             color=plot_colour, alpha=0.75,
             zorder=1.2)

ax1.axhline(y=driving_field_freq * 1e-9,
            lw=2, ls='-',
            color='blue', alpha=0.5,
            label=r'$f_{d}$:' + f'{driving_field_freq*1e-9: .2f}',
            zorder=1.98)

ax1.set(title='Dispersion relation',
        xlabel=r'Wavevector, $k$ (nm$^{-1}$)',
        ylabel=r'Frequency, $f$ (GHz)',
        xlim=(-0.15, 0.15),
        ylim=(0, 40))

ax1.legend(labelcolor='black')

plt.show()

# 5. Relaxation

## 5.1. Drive the system

### 5.1.1. Define the minimisation evolver

In [ ]:
min_evolver = oc.CGEvolver(method='Polak-Ribiere',
                           fixed_spins=evolver_fixed_spins_list,
                           #energy_precision=1e-10  # May need to increase for large systems
                           )

minD = oc.MinDriver(evolver=min_evolver,
                    stopping_mxHxm=0.1
                    )

### 5.1.2. Apply the `TimeDriver`

In [ ]:
minD.drive(system,
           dirname=data_output,
           fixed_subregions=evolver_fixed_spins_list,
           verbose=2
           )

## 5.2. Display and save the output

In [ ]:
system_region = csp.MyRegions(system.name + "_regions")

system_region.main(p1=tuple(main_region.pmin),
                   p2=tuple(main_region.pmax),
                   dims=tuple(main_region.dims),
                   units=tuple(main_region.units)
                   )

system_region.mesh = system.m.mesh

system_prop = csp.SystemProperties(length=main_region.pmax[0],
                                   width=main_region.pmax[1],
                                   thickness=main_region.pmax[2])

# System mesh cell size
system_prop.cell = tuple(main_cellsize)
system_prop.units = ('m', 'm', 'm')
system_prop.update_numcells()

In [ ]:
%%capture
cip.default_three_pane(system=system,
                       system_prop=system_prop,
                       system_region=system_region,
                       figs_output_dir=figs_output,
                       drive_number=system.drive_number,
                       fig_name='after_relaxation',
                       has_schematic=False,
                       #scalar_resample_shape=[256, 3],
                       # vector_resample_shape=[16, 3]
                       )

# 6. Drive the system with the time-dependent field

## 6.1. Set temporal parameters

In [ ]:
# Maximum time for simulation
tmax = 10e-9  # [s]

# num_datapoints = round(tmax / stepsize)  # Warning: can lead to very large output files!
num_datapoints = 200

# Spacing between saved outputs
stepsize_between_datapoints = tmax / num_datapoints

## 6.2. Add time-dependent fields to the energy equation

In [ ]:
if energy_model.driving_zeeman:
    system.energy += energy_zeeman_drive

### 6.2.1. Check the energy equation

In [ ]:
system.energy

## 6.3. Drive the system

### 6.3.1. Define the evolver

In [ ]:
sim_evolver = oc.RungeKuttaEvolver(method='rkf54',
                                   #alpha=alpha,  # Do not supply `alpha` here if already defined in `system.dynamics`
                                   #gamma_G=gamma0*mm.consts.mu0,
                                   # Driver may override timestep conditions if stopping_criteria are given
                                   #min_timestep=0,  # Default: zero.
                                   #max_timestep=stepsize,  # Default: 1e-10 [s].
                                   fixed_spins=evolver_fixed_spins_list,
                                   do_precess=1,
                                   #energy_precision=1e-10,  # May need to increase for large systems
                                   allow_signed_gamma=0,
                                   )

td = oc.TimeDriver(evolver=sim_evolver)

### 6.3.2. Apply the `TimeDriver`

It's at this point we pass our JSON parameters to the driver.

In [ ]:
td.drive(system,
         t=tmax,
         dirname=data_output,
         n=num_datapoints,
         n_threads=10,
         fixed_subregions=evolver_fixed_spins_list,
         verbose=2,
         **json_params
         )

## 6.4. Display and save the output

In [ ]:
cip.default_three_pane(system=system,
                       system_prop=system_prop,
                       system_region=system_region,
                       figs_output_dir=figs_output,
                       drive_number=system.drive_number - 1,
                       fig_name=f'driven_{system.drive_number - 1}',
                       has_schematic=False)

## 6.5. Breakpoint

Pause here to check the initial data outputs before completing more advanced post-processing.

This is also an appropriate place to add temporary debug lines to the code. Note that these should be deleted after debugging is complete.

In [ ]:
clicked_wait_button = False
wait_button_output = widgets.Output()

@wait_button_output.capture(clear_output=True, wait=True)
def _on_click(btn):
    global clicked_wait_button
    clicked_wait_button = True
    print('Moving on to post-processing')

run_button = widgets.Button(
    description='Press to continue',
    button_style='info'
)
run_button.on_click(_on_click)
display(run_button, wait_button_output)


# Block here, process widget messages, but stop after 60 s
timeout = 60  # seconds
start = time.monotonic()
with ui_events() as poll:
    while not clicked_wait_button:
        if time.monotonic() - start > timeout:
            raise KeyboardInterrupt("User did not press the button. Kernel is still live.")
        poll(2)       # handle up to 2 pending UI events
        time.sleep(0.5)

del clicked_wait_button

# 7. Post-processing

## 7.1. Import data

In [ ]:
try:
    system.name
except NameError:
    system = mm.System(name='dmi_paper_validation')
    system.drive_number = 4

In [ ]:
m_all_data = md.Drive(name=system.name, dirname=data_output,
                      number=system.drive_number - 1, use_cache=True)

## 7.2. Extract required values while handling NameErrors

In [ ]:
try:
    system_prop
except NameError:

    system_prop = csp.SystemProperties(length=m_all_data.m0.mesh.region.pmax[0],
                                       width=m_all_data.m0.mesh.region.pmax[1],
                                       thickness=m_all_data.m0.mesh.region.pmax[2])

    # System mesh cell size
    system_prop.cell = tuple(m_all_data.m0.mesh.cell)
    system_prop.units = m_all_data.m0.mesh.region.units
    system_prop.update_numcells()

try:
    sat_mag
except NameError:
    sat_mag = float(m_all_data.info['Saturation fields [A/m]'])

try:
    stepsize_between_datapoints
except NameError:
    stepsize_between_datapoints = m_all_data.table.data.t[0]


In [ ]:
data_m0 = m_all_data.m0.to_xarray().values

m_pos = [int(round(system_prop.width * 0.5 / system_prop.cell[1], 0)),
         int(round(system_prop.thickness * 0.5 / system_prop.cell[2], 0))]

data_mx0 = data_m0[:, m_pos[0], m_pos[1], 0] / sat_mag
data_my0 = data_m0[:, m_pos[0], m_pos[1], 1] / sat_mag
data_mz0 = data_m0[:, m_pos[0], m_pos[1], 2] / sat_mag

data_mx = np.zeros((m_all_data.n, int(system_prop.numcells[0])))
data_my = np.zeros((m_all_data.n, int(system_prop.numcells[0])))
data_mz = np.zeros((m_all_data.n, int(system_prop.numcells[0])))

m_data = m_all_data.to_xarray()

for i, _ in enumerate(m_data):
    data_mx[i] = (m_data[i, :, m_pos[0], m_pos[1], 0].values / sat_mag) - data_mx0
    data_my[i] = (m_data[i, :, m_pos[0], m_pos[1], 1].values / sat_mag) - data_my0
    data_mz[i] = (m_data[i, :, m_pos[0], m_pos[1], 2].values / sat_mag) - data_mz0

In [ ]:
%%capture
# Prepare the meshgrid for 3D plot
x = np.linspace(0, system_prop.length, data_mx.shape[1])
z = np.linspace(0, system_prop.thickness, data_mx.shape[0])
X, Z = np.meshgrid(x, z)
Y = np.ones_like(X) * system_prop.width / 2  # y is fixed slice

# Create a 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
surf = ax.plot_surface(X, Y,
                       data_mx,
                       cmap='viridis')  # data_mx as example

# Add color bar to indicate the scale
fig.colorbar(surf)

# Customize the view angle
ax.view_init(elev=30, azim=120)

# Set labels
ax.set_xlabel('Length')
ax.set_ylabel('Width')
ax.set_zlabel('Magnetization')

## 7.3. Temporal evolution of the system

In [ ]:
time_index = min(100, data_mx.shape[0])
plt.plot(np.arange(0, system_prop.numcells[0] * (system_prop.cell[0] / 1e-9), system_prop.cell[0] / 1e-9),
         data_mx[time_index-1, :], alpha=0.8, color='C1', label='OOMMF')
#my_pos_times, y_pos_vals = np.loadtxt('/Users/cameronmceleney/pCloud Drive/test/data_se.txt')
#plt.plot(my_pos_times, my_pos_vals[::-1], zorder=0.9, label='C++', color='C4')
plt.gca().set(title=f'Drive: {m_all_data.info['drive_number']} | Time: {time_index * stepsize_between_datapoints / 1e-9: .3f} ns',
              xlim=(1, system_prop.length * 1e9),
              #ylim=(-0.002, 0.002),
              xlabel='Distance (nm)', ylabel='Amplitude (a.u.)')
plt.legend()
plt.savefig(figs_output + '/drive-' + str(m_all_data.number) + f'_pos_{int(time_index * stepsize_between_datapoints * 1e9)}ns.png',
            dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
 # Cell which acts as a breakpoint; 0/0 intentionally produces an error. Alternative is to utilise the 'raise' keyword
#raise ValueError('Breakpoint')

## 7.4. Spatial evolution of the system

In [ ]:
pos_index_1 = 4000
pos_index_2 = system_prop.numcells[0] - 1 - pos_index_1
plt.plot(np.linspace(0, tmax-stepsize, num_datapoints)*1e9, data_mx[:, pos_index_1], alpha=0.8, color='C1', label=f'{pos_index_1: .0f}')
plt.plot(np.linspace(0, tmax-stepsize, num_datapoints)*1e9, data_mx[:, pos_index_2], alpha=0.8, color='C4', label=f'{pos_index_2: .0f}')

#my_times, my_vals = np.loadtxt('/Users/cameronmceleney/pCloud Drive/test/data.txt')
#plt.plot(my_times[:], my_vals, zorder=0.9, color='C4', label='C++')
plt.gca().set(title=f'Drive: {m_all_data.number}', xlim=(0, 5),
              xlabel='Time (ns)', ylabel='Amplitude (a.u.)')
plt.legend(title='Site from\nLHS (nm)')

plt.savefig(figs_output + '/drive-' + str(m_all_data.number) + f'_time_{pos_index_1}nm.png', dpi=300, bbox_inches='tight')

## Dispersion relation

In [ ]:
def Omega_generalised_with_ua(system_prop, H0, Ms, A, D, k, d, K1, K2, aniso_axis, gamma, p=1,
                              has_demag=1, has_dmi=1, has_aniso=1):
    J = 2 * A / (mm.consts.mu0 * Ms)
    DM = -2 * D / (mm.consts.mu0 * Ms)

    demag_factors = cpe.calculate_demag_factor_uniform_prism(system_prop.length - 2 * d,
                                                             system_prop.width,
                                                             system_prop.thickness)
    # print(demag_factors)

    #om = np.sqrt((H0 + J * (k ** 2) + has_demag * Ms * (demag_factors['N_x'] - demag_factors['N_z']))
    #             * (H0 + J * (k ** 2) + has_aniso * ((4 * (aniso_axis[2] ** 2)) / (Ms * mm.consts.mu0)
    #                                                 * (1 * K1 + 2 * K2 * (aniso_axis[2] ** 2)))
    #                + has_demag * Ms * (demag_factors['N_y'] - demag_factors['N_z'])
    #                )
    #             + has_aniso * (4 * (aniso_axis[2] ** 4)) / (Ms ** 2 * mm.consts.mu0 ** 2)
    #             * (4 * (K1**2) + 4 * K1 * K2 * (aniso_axis[2] ** 2) + 4 * (K2 ** 2) * (aniso_axis[2] ** 4))
    #             )
    om = np.sqrt((H0
                  + J * (k ** 2)
                  + has_aniso * ((2 * (aniso_axis[2] ** 2)) / (Ms * mm.consts.mu0) * (1 * K1 + 2 * K2 * (aniso_axis[2] ** 2)))
                  + has_demag * Ms * (demag_factors['N_x'] - demag_factors['N_z']))
                 * (H0
                    + J * (k ** 2)
                    + has_aniso * ((2 * (aniso_axis[2] ** 2)) / (Ms * mm.consts.mu0) * (1 * K1 + 2 * K2 * (aniso_axis[2] ** 2)))
                    + has_demag * Ms * (demag_factors['N_y'] - demag_factors['N_z'])
                )
             )

    om += has_dmi * p * DM * k

    # the mu0 factor shown in the paper is not necessary if we use gamma
    # in Hz / (A / m)
    om *= (gamma * mm.consts.mu0)

    return om

In [ ]:
kmax = 0.8 * 1e9
ks = np.linspace(1e-10, kmax, int(round(system_prop.numcells[0] / 2)))
ks_n = np.linspace(-kmax, 1e-10, int(round(system_prop.numcells[0] / 2)))

In [ ]:

# Moon Eq. (12)
#oms = cdr.Omega_Moon(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)
#oms_n = cdr.Omega_Moon(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks_n, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)
#
## Moon Eq. (14)
#oms_l = cdr.Omega_Moon_large_k(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)
#oms_l_n = cdr.Omega_Moon_large_k(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks_n, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)
#
## Moon Eq. (16)
#oms_s = cdr.Omega_Moon_small_k(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)
#oms_s_n = cdr.Omega_Moon_small_k(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks_n, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)
#
## Custom dispersion relation
#oms_c = cdr.Omega_Moon_custom(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)
#oms_c_n = cdr.Omega_Moon_custom(zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks_n, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, demag=1, has_dmi=1)

In [ ]:
# Custom dispersion relation
oms_g = cdr.Omega_generalised_with_ua(system_prop,
                                         zeeman_static_strength[2], sat_mag, exchange_stiffness, D_ij,
                                         ks, system_prop.cell[0],
                                         K_1, K_2, aniso_axis,
                                         gamma0 / (2 * np.pi * 1e9), p=1,
                                         has_demag=1, has_dmi=1, has_aniso=1)
oms_g_n = cdr.Omega_generalised_with_ua(system_prop,
                                           zeeman_static_strength[2], sat_mag, exchange_stiffness, D_ij,
                                           ks_n, system_prop.cell[0],
                                           K_1, K_2, aniso_axis,
                                           gamma0 / (2 * np.pi * 1e9), p=1,
                                           has_demag=1, has_dmi=1, has_aniso=1)

times = m_data['t'].values
x = m_data['x'].values

# Create Hanning windows
times_filter = np.hanning(len(times))
x_filter = np.hanning(len(x))

# Apply the Hanning window to the data
window = np.outer(times_filter, x_filter)
data_mx_windowed = data_mx * window

fft_data = np.fft.fft2(data_mx_windowed)
fft_data = abs(np.fft.fftshift(fft_data))
log_fft_data = np.log10(fft_data**2)

freqs = np.fft.fftfreq(n=len(times), d=(times[1] - times[0])) * 1e-9
freqs = np.fft.fftshift(freqs)

k = np.fft.fftfreq(n=len(x), d=(x[1] - x[0])) * 1e-9
k = np.fft.fftshift(k) * 2 * np.pi

fig5, ax = plt.subplots(nrows=1, ncols=1)
ax.imshow(log_fft_data, extent=[k[0], k[-1], freqs[0], freqs[-1]],
          vmin=log_fft_data.min() / 10, vmax=log_fft_data.max(),
          aspect='auto', interpolation='none',
          cmap='magma')

#ax.plot(ks * 1e-9, oms, lw=2, ls='--', color='white', alpha=0.5, label='Theory (Moon)')
#ax.plot(ks_n * 1e-9, oms_n, lw=2, ls='--', alpha=0.5, color='white')

#ax.plot(ks * 1e-9, oms_l, lw=2, ls='-.', color='lightgreen', alpha=0.5, label='Theory (Large k)')
#ax.plot(ks_n * 1e-9, oms_l_n, lw=2, ls='-.', alpha=0.5, color='lightgreen')

#ax.plot(ks * 1e-9, oms_s, lw=2, ls='-.', color='orange', alpha=1, label='Theory (Small k)')
#ax.plot(ks_n * 1e-9, oms_s_n, lw=2, ls='-.', alpha=1, color='orange')

#ax.plot(ks * 1e-9, oms_c, lw=2, ls=':', color='lightblue', alpha=1, label='Theory (Custom)')
#ax.plot(ks_n * 1e-9, oms_c_n, lw=2, ls='-.', alpha=1, color='lightblue')

ax.plot(ks * 1e-9, oms_g, lw=2, ls='-.', color='yellow', alpha=0.75, label='Theory (Ubermag)')
ax.plot(ks_n * 1e-9, oms_g_n, lw=2, ls='-.', alpha=0.75, color='yellow')

# Load the arrays from the .npz file
#data = np.load('/Users/cameronmceleney/Data/2024-07-29/data.npz')
#ksc = data['ks']
#ksc_n = data['ks_n']
#omsc_g = data['oms_g']
#omsc_g_n = data['oms_g_n']

#ax.plot(ksc * 1e-9, omsc_g, ls='-.', color='purple', alpha=0.75, label='Theory (C++)')
#ax.plot(ksc_n * 1e-9, omsc_g_n, ls='-.', alpha=0.75, color='purple')

ax.set(xlabel=r'Wavevector, $k$ (nm$^{-1}$)', ylabel=r'Frequency, $f$ (GHz)',
       xlim=(-0.15, 0.15),
       ylim=(0, 40)
       )


#handles, labels = ax.get_legend_handles_labels()
#blacks = matplotlib.patches.Patch(color='purple', label='OOMMf')
#ax.legend(handles=[handles[0], blacks])
ax.legend(labelcolor='white')
plt.savefig(figs_output + '/drive-' + str(system.drive_number - 1) + '_dispersion_relation.png', dpi=300, bbox_inches='tight')


In [ ]:
fig_fft, fig_fft_axes = plt.subplot_mosaic(mosaic=[['k-space', 'k-space'],
                                          ['freq-space-lhs', 'freq-space-rhs']],
                                  figsize=(12, 8),
                                  gridspec_kw={'width_ratios': (1, 1),
                                               'height_ratios': (1, 1),
                                               'wspace': 0.0,
                                               'hspace': 0.0},
                                  layout='constrained',
                                  facecolor='lightgrey'
                                  )

fig_fft.suptitle(f'Drive: {m_all_data.number}', fontsize=20)

for i in range(1, int(tmax * 1e9)+1):
    fig_fft_axes['k-space'].plot(k, log_fft_data[i * 1000 - 1, :], label=f'{i}')
fig_fft_axes['k-space'].set(title=f'FFT across full spin chain',
              xlabel='Wavevector',
              ylabel='Amplitude (log)',
              xlim=[-0.2, 0.2])
fig_fft_axes['k-space'].legend(title='Sim. time (ns)', ncols=2)

for fft_pos_index in range(300, 2101, 300):
     fig_fft_axes['freq-space-lhs'].plot(freqs, log_fft_data[:, fft_pos_index], #label=f'{fft_pos_index * system_prop.cell[0] * 1e9 * 2: .0f}')
                                         label=f'{fft_pos_index * system_prop.cell[0] * 1e9: .0f}')
fig_fft_axes['freq-space-lhs'].set(title=f'FFT at single site',
              xlabel='Frequency (GHz)',
              ylabel='Amplitude (log)',
              xlim=[0, 40])
fig_fft_axes['freq-space-lhs'].axvline(x=driving_field_freq*1e-9, color='black', zorder=1.1, alpha=0.75,
                                       label=f'Driving freq: {driving_field_freq*1e-9:.2f} GHz')

fig_fft_axes['freq-space-lhs'].legend(title='Site (nm)', ncols=2, loc='upper right')


for fft_pos_index in range(4100, 2299, -300):
    fig_fft_axes['freq-space-rhs'].plot(freqs, log_fft_data[:, fft_pos_index], #label=f'{fft_pos_index * system_prop.cell[0] * 1e9 * 2: .0f}')
                                        label=f'{fft_pos_index * system_prop.cell[0] * 1e9: .0f}')
fig_fft_axes['freq-space-rhs'].set(title=f'FFT at single site',
              xlabel='Frequency (GHz)',
              ylabel='Amplitude (log)',
              xlim=[0, 40])
fig_fft_axes['freq-space-rhs'].axvline(x=driving_field_freq*1e-9, color='black', zorder=1.1, alpha=0.75,
                                        label=f'Driving freq: {driving_field_freq*1e-9:.2f} GHz')
fig_fft_axes['freq-space-rhs'].legend(title='Site (nm)', ncols=2, loc='upper right')

plt.savefig(figs_output + '/drive-' + str(m_all_data.number) + '_fft_mosaic.png', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), nrows=1, ncols=1)
#for fft_pos_index in range(300, 2101, 300):
for fft_pos_index in [4402]:
     ax.plot(k, log_fft_data[time_index-1, :], #label=f'{fft_pos_index * system_prop.cell[0] * 1e9 * 2: .0f}')
                                         label=f'left')
ax.set(title=f'FFT at single site',
              xlabel='Wavevector (nm^-1)',
              ylabel='Amplitude (log)',
       )



ax.legend(title='Site (nm)', ncols=1)

In [ ]:
import scipy as sp
import numpy as np

def _fft_data(input_data, spatial_spacing: bool | float = None, fft_window: str = False):
    """
    Computes the discrete Fourier transform (DFT) of a given 1-D signal using FFT algorithms.

    Regarding the returned `dft_samples`, if the `input_data` contains:
        - spatial information then dft_samples will represent 'wavevectors', and be in units of inverse-length.
        - temporal information then dft_samples will represent 'frequencies', and be in units of Hz.

    Regarding the returned `discrete_fourier_transform`, it will contain the magnitudes of the DFT results. It will
    be the absolute value of the DFT results; caution if the phase information is needed for subsequent analysis.

    :param input_data: Magnitudes of a magnetic moment components for a given axis.
    :param spatial_spacing: The lattice constant for the system for spatial data.
    :param fft_window: Apply a window to the data before taking the FFT; default is a Hann window.

    :return: A tuple containing the sample frequencies [0] and DFT results [1].
    """

    # Pad the data to greatly improve efficiency of the FFT computation
    n_total = input_data.size

    n_total_padded = sp.fft.next_fast_len(n_total)

    # Find the bin size
    if spatial_spacing is None:
        # For temporal: divide the simulated time into equally sized segments based upon the number of data-points.
        sample_spacing = (tmax / (num_datapoints - 1))
    else:
        # For spatial: division of lattice is already known with the lattice constant
        sample_spacing = spatial_spacing

    # Data for DFT (create copy to avoid modifying original data)
    data_to_process = input_data

    # Select a window for the FFT.
    if fft_window is not None:
        if isinstance(fft_window, str):
            # For valid windows, see https://docs.scipy.org/doc/scipy/reference/signal.windows.html
            if fft_window == 'bb':
                window = 1
            else:
                window_func = getattr(sp.signal.windows, fft_window, "hann")
                window = window_func(n_total)
        elif callable(fft_window):
            # Incase the user wants a custom window function
            window = sp.signal.fft_window(n_total)
        else:
            # Default case
            window = sp.signal.windows.hann(n_total)

        # *= doesn't work here for some reason without breaking subplots
        data_to_process = data_to_process * window

    # Perform the FFT
    discrete_fourier_transform = sp.fft.fft(data_to_process, n_total_padded)

    # Samples from the DFT.
    dft_samples = sp.fft.fftfreq(n_total_padded, sample_spacing)

    # Always skip the DC component at y[0] as I don't need the signal's mean value
    if n_total_padded % 2 == 0:
        # For N even, the elements `y = [1, N / 2 -1]` contain the positive-frequency terms with the final
        # element `y = N / 2` containing the Nyquist frequency
        positive_wavevector_indices = slice(1, n_total_padded // 2)
    else:
        # For N odd, the elements `y = [1, (N - 1) / 2]` contain the positive-frequency terms
        positive_wavevector_indices = slice(1, (n_total_padded + 1) // 2)

    # Only want positive wavevectors and their corresponding fourier transform. Must take absolute value of DFT.
    discrete_fourier_transform = np.abs(discrete_fourier_transform[positive_wavevector_indices])
    dft_samples = dft_samples[positive_wavevector_indices]

    return dft_samples, discrete_fourier_transform

In [ ]:
cdr.Omega_generalised_with_ua(system_prop,
                              zeeman_static_strength[2], sat_mag, exchange_stiffness, D_ij,
                              0.075e9, system_prop.cell[0],
                              K_1, K_2, aniso_axis,
                              gamma0 / (2 * np.pi * 1e9), p=1,
                              has_demag=1, has_dmi=1, has_aniso=1)

In [ ]:
#k_lines_arr = np.array([ 0.0517, -0.0334,  0.0736, -0.0522,  0.0978, -0.0734,  0.1053,
#       -0.0802,  0.1261, -0.0993,  0.1317, -0.1046,  0.1554, -0.1271,
#        0.1807, -0.1516,  0.1857, -0.1564,  0.2099, -0.1802,  0.2331,
#       -0.2031,  0.2404, -0.2103,])
#k_lines_arr_bal = np.array([
#    0.06400, -0.04390, 0.08540, -0.06240, 0.12120, -0.09480, 0.17430, -0.14530, 0.19580, -0.16640,
#    0.22820, -0.19820,
#])
#k_lines_unknown = np.array([
#    0.04, 0.14
#])

In [ ]:
#k_lines_arr = np.array(
#    [
#    #0.07300, -0.05160,
#	#0.09670, -0.07240,
#	#0.12570, -0.09890,
#	#0.15720,  -0.12880,
#	#0.18070,  -0.15160,
#	#0.20990,  -0.18020,
#	#0.23070,  -0.20070,
#])
#k_lines_arr_bal = np.array([
#                            #0.07295, -0.05160,
#                            #0.10450, -0.07945,
#                            #0.13205, -0.10490,
#                            #0.15720, -0.12880,
#                            #0.18840, -0.15910,
#                            #0.20990, -0.18015,
#                            #0.24035, -0.21025,
#])
#k_lines_unknown = np.array([
#                            0.04000, -0.02350,
#                            0.05960, -0.04020,
#                            0.09010, -0.06650,
#                            0.12120, -0.09480,
#                            0.14060,  -0.11300,
#                            0.17425, -0.14530,
#                            0.19290,  -0.16350,
#                            0.22570,  -0.19570,
#                            ])

In [ ]:
#_lines_arr = np.array([
#                       #0.03075, #-0.01550,
#                       #0.05080, #-0.03270,
#                       #0.07935, #-0.05710,
#                       #0.10450, #-0.07945,
#                       #0.12780, #-0.10095,
#                       #0.15720, #-0.12880,
#                       #0.18065, #-0.15155,
#                       #0.20990, #-0.18015,
#                       #0.23310, #-0.20310,
#                       -0.109, 0.129  # 22.5 GHz
#)
#_lines_arr_bal = np.array([
#                           #0.03075, #-0.01550,
#                           #0.06255, #-0.04265,
#                           #0.08830, #-0.06500,
#                           #0.12120, #-0.09480,
#                           #0.14985, #-0.12180,
#                           #0.17590, #-0.14690,
#                           #0.20015, #-0.17060,
#                           #0.23065, #-0.20065,
#                           #0.25870, #-0.22845,
#                           -0.129, 0.157  # 30GHz
#)
#_lines_unknown = np.array([
#                          # 0.063, -0.04 , # 10
#                          # 0.169, -0.14, # 33
#                          # 0.04,  -0.024, # 7.8
#                          # 0.14,  -0.113, # 25.45
#                           #-0.04, 0.059, # 10
#                           #-0.14, 0.169, # 33
#                           #0.04, -0.024, # 7.8
#                           #0.14, -0.113, # 25.45
#                           #0.18, -0.15, # 36.6
#                           #0.094, -0.74, # 15.4

#                           #0.13, 0.124,#?
#                           #-0.103, 0.158
#                           -0.079, 0.109 , # 17.6
#   0.04


#                                       ])

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(10, 3), nrows=1, ncols=2)
fig.suptitle(f'Drive: {m_all_data.number}', fontsize=20)
freq_loc_1 = pos_index_1
freq_loc_2 = pos_index_2
frequencies1, fourier_transform_freq1 = _fft_data(input_data=data_mx[:, freq_loc_1],
                                    fft_window='bb')
frequencies2, fourier_transform_freq2 = _fft_data(input_data=data_mx[:, freq_loc_2],
                                     fft_window='bb')
frequencies1 *= 1e-9
frequencies2 *= 1e-9

#for fft_pos_index in range(300, 2101, 300):
ax1.plot(frequencies1, fourier_transform_freq1,
       label=f'Site: {freq_loc_1}')

ax1.plot(frequencies2, fourier_transform_freq2,
         label=f'Site: {freq_loc_2}')

ax1.set(title=f'FFT at single site',
              xlabel='Frequency (GHz)',
              ylabel='Amplitude (log)',
              xlim=[0, 40])

ax1.axvline(x=driving_field_freq*1e-9, color='black', zorder=1.1, alpha=0.75,
            label=f'f_drive: {driving_field_freq*1e-9:.2f} GHz')


#for fft_pos_index in range(4100, 2299, -300):


ax1.legend(title='Site (nm)', ncols=1)

###
wavevectors1, fourier_transform_k1 = _fft_data(input_data=data_mx[time_index-1, 600:4002],
                                    spatial_spacing=1e-9, fft_window='bb')
wavevectors2, fourier_transform_k2 = _fft_data(input_data=data_mx[time_index-1, 4802:8204],
                                    spatial_spacing=1e-9, fft_window='bb')
wavevectors1 *= 1e-9 * 2 * np.pi
wavevectors2 *= 1e-9 * 2 * np.pi

ax2.plot(wavevectors1, fourier_transform_k1,
           lw=1.5, marker='', markerfacecolor='black', markeredgecolor='black',
           label=f"Left of drive - no dmi")
ax2.plot(wavevectors2, fourier_transform_k2,
           lw=1.5, marker='', markerfacecolor='black', markeredgecolor='black',
           label=f"Right of drive - no dmi")
ax2.set(title="FFT across whole chain",
    xlabel=f"Wavevector, $k$", ylabel=f"Intensity (a.u.)", yscale='log',
       xlim=(0, 0.3))
#ax.axvline(x=9.2)
ax2.legend()

In [ ]:
# Cell which acts as a breakpoint; 0/0 intentionally produces an error. Alternative is to utilise the 'raise' keyword
raise ValueError('Breakpoint')

In [ ]:
kmax = 0.8 * 1e9
ks = np.linspace(1e-10, kmax, int(round(system_prop.numcells[0] / 2)))
ks_n = np.linspace(-kmax, 1e-10, int(round(system_prop.numcells[0] / 2)))


oms_g = Omega_generalised(system_prop, zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, has_demag=1, has_dmi=1)
oms_g_n = Omega_generalised(system_prop, zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks_n, 1e-9, gamma0 / (2 * np.pi * 1e9), p=1, has_demag=1, has_dmi=1)

# Custom dispersion relation
oms_g_ua = cdr.Omega_generalised_with_ua(system_prop, zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks,
                                         1e-9, K_1, K_2, aniso_axis,
                                         gamma0 / (2 * np.pi * 1e9), p=1, has_demag=1, has_dmi=1, has_aniso=1)
oms_g_ua_n = cdr.Omega_generalised_with_ua(system_prop, zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks_n,
                                       1e-9, K_1, K_2, aniso_axis, gamma0 / (2 * np.pi * 1e9), p=1, has_demag=1, has_dmi=1, has_aniso=1)

oms_g_n_ua = cdr.Omega_generalised_with_ua(system_prop, zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks,
                                     1e-9, 0, 0, aniso_axis, gamma0 / (2 * np.pi * 1e9), p=1, has_demag=1, has_dmi=1, has_aniso=1)
oms_g_n_ua_n = cdr.Omega_generalised_with_ua(system_prop, zeeman_static[2], sat_mag, exchange_stiffness, D_ij, ks_n,
                                       1e-9, 0, 0, aniso_axis, gamma0 / (2 * np.pi * 1e9), p=1, has_demag=1, has_dmi=1, has_aniso=1)

fig5, ax = plt.subplots(nrows=1, ncols=1)

ax.plot(ks * 1e-9, oms_g_ua, lw=2, ls='-', color='purple', alpha=0.75,
        label='Theory', zorder=1.99)
ax.plot(ks_n * 1e-9, oms_g_ua_n, lw=2, ls='-', alpha=0.75, color='purple',
        zorder=1.99)

#ax.plot(ks * 1e-9, oms_g_n_ua, lw=2, ls='--', color='red', alpha=0.75,
#        label='Theory (New equation - no anisotropy)', zorder=1.98)
#ax.plot(ks_n * 1e-9, oms_g_n_ua_n, lw=2, ls='--', alpha=0.75, color='red',
#        zorder=1.99)

#ax.plot(ks * 1e-9, oms_g, lw=2, ls='-', color='blue', alpha=0.75,
#        label='Theory (Old equation - no anisotropy)', zorder=1.97)
#ax.plot(ks_n * 1e-9, oms_g_n, lw=2, ls='-', alpha=0.75, color='blue',
#        zorder=1.97)

ax.set(xlabel=r'Wavevector, $k$ (nm$^{-1}$)', ylabel=r'Frequency, $f$ (GHz)',
       xlim=(-0.15, 0.15),
       ylim=(0, 40)
       )
ax.legend(labelcolor='black')

plt.show()